In [1]:
from dataclasses import asdict
from threading import Event, Lock, Semaphore
from typing import Dict, List, Tuple, Any
from functools import partial, update_wrapper
from collections import defaultdict
from pathlib import Path
from datetime import datetime
from queue import Queue
import argparse
import logging
import hashlib
import json
import sys
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import proxystore as ps
from proxystore.store import register_store
from proxystore.store.file import FileStore
from proxystore.store.globus import GlobusEndpoints, GlobusStore
from proxystore.store.redis import RedisStore
from proxystore.store.utils import get_key
from rdkit import Chem
from tqdm import tqdm
from colmena.models import Result
from colmena.queue.redis import RedisQueues
from colmena.thinker import BaseThinker, result_processor, event_responder, task_submitter, agent
from colmena.thinker.resources import ResourceCounter
from colmena.task_server import ParslTaskServer
from qcelemental.models import OptimizationResult, AtomicResult

from moldesign.score.nfp import evaluate_mpnn, retrain_mpnn, NFPMessage, custom_objects
from moldesign.store.models import MoleculeData
from moldesign.store.recipes import apply_recipes
from moldesign.utils import get_platform_info

# from config import theta_debug_and_venti as make_config
from config import csecluster_RT_scale as make_config

2023-10-07 16:19:31.875014: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 16:19:32.830790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
model_path = '/home/yxx/work/project/colmena/multisite_/data/moldesign/initial-model/networks'
paths = os.listdir(model_path)
models = [
tf.keras.models.load_model(os.path.join(model_path,path,"model.h5") , custom_objects=custom_objects)
for path in tqdm(paths[:2], desc='Loading models')
]

Loading models:   0%|          | 0/2 [00:00<?, ?it/s]

Loading models:  50%|█████     | 1/2 [00:00<00:00,  1.81it/s]

Loading models: 100%|██████████| 2/2 [00:02<00:00,  1.27s/it]


In [6]:
with open('/home/yxx/work/project/colmena/multisite_/data/moldesign/training-data.json') as fp:
    database = [MoleculeData.parse_raw(line) for line in fp]
property_name = 'xtb-vacuum'
train_data = dict(
            (d.identifier['smiles'], d.oxidation_potential[property_name])
            for d in database
        )
model = models[0]


In [15]:
retrain_mpnn = partial(retrain_mpnn, num_epochs=128, learning_rate=1e-4, timeout=600, device_type='gpu')
my_retrain_mpnn = update_wrapper(retrain_mpnn, retrain_mpnn)
my_retrain_mpnn(model.get_config(), train_data)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2023-10-07 16:44:26.243084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-07 16:44:26.810652: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020TensorDataset:96"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_

FailedPreconditionError: Graph execution error:

Detected at node 'Adam/update_0_63/StatefulPartitionedCall' defined at (most recent call last):
    File "/home/yxx/miniconda3/envs/multisite/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/yxx/miniconda3/envs/multisite/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/yxx/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/yxx/miniconda3/envs/multisite/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/yxx/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/yxx/.local/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/yxx/min